In [1]:
from divar import client
import requests
import pandas as pd
from tqdm import tqdm

divar = client.Divar()
divar.post('wYvPZ336')

{'title': 'سالم',
 'description': 'برای بازدید هماهنگ شود',
 'web_url': 'https://divar.ir/v/wYvPZ336',
 'price': 1800000,
 'rent': None,
 'credit': None,
 'category': 'carpet',
 'business_type': 'personal',
 'images': ['https://s100.divarcdn.com/static/pictures/1676679996/wYvPZ336.webp'],
 'city': 'تهران',
 'district': 'ارامنه'}

In [2]:
def get_tokens(last_post_date, city_number, post_category='apartment-sell', n_pages=4):
   
    url = 'https://api.divar.ir/v8/web-search/{city_number}/{post_category}'.format(
        city_number=city_number,
        post_category=post_category
    )

    headers = {
        'Content-Type': 'application/json'
    }

    list_of_tokens = []
    for i in range(n_pages):
        json = {"json_schema": {"category": {"value": post_category}},
                "last-post-date": last_post_date}
        res = requests.post(url, json=json, headers=headers)
        
        data = res.json()
        last_post_date = data['last_post_date']

        for widget in data['web_widgets']['post_list']:
            token = widget['data']['token']
            list_of_tokens.append(token)

    return list_of_tokens


In [3]:
categories =['apartment-sell','vehicles','electronic-devices','home-kitchen','entertainment','social-services','tools-materials-equipment']

In [6]:
for category in categories:
    
    print(category)
    
    token_lists = get_tokens(last_post_date = 0,city_number = 1, post_category = category)
    #len(token_lists)
    posts_list = []
    
    for token in tqdm(token_lists):
        posts_list.append(divar.post(token))
        
    df = pd.DataFrame.from_dict(posts_list)
    df['token'] = df['web_url'].str.split('https://divar.ir/v/', expand=True)[1]
    
    df.to_csv('data/'+category+'/datas.csv')
    
    for token in tqdm(token_lists):
    
        post = divar.post(token)
        images = post['images']
        description = post['description']
        i = 0
        for img in tqdm(images):
            i = i+1
            pic_url  = img

            with open('data/'+category+'/images/'+token + '_' + str(i)+ '.jpg', 'wb') as handle:

                response = requests.get(pic_url, stream=True)

                if not response.ok:
                    print(response)

                for block in response.iter_content(1024):
                    if not block:
                        break

                    handle.write(block)
        
    

tools-materials-equipment


  7%|███▏                                        | 7/96 [00:14<03:03,  2.06s/it]
0it [00:00, ?it/s]
 20%|████████▌                                  | 19/96 [00:30<02:06,  1.65s/it]


  0%|                                                     | 0/4 [00:00<?, ?it/s]


 46%|███████████████████▋                       | 44/96 [01:40<02:54,  3.35s/it]


 61%|██████████████████████████▍                | 59/96 [02:28<01:11,  1.93s/it]


 67%|██████████████████████████████               | 2/3 [00:00<00:00,  3.84it/s]


 20%|█████████                                    | 1/5 [00:01<00:04,  1.03s/it]


100%|███████████████████████████████████████████| 96/96 [03:37<00:00,  2.27s/it]


In [45]:
# from PIL import Image
# image = Image.open('pic1.jpg')
# image.show()
